# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
opened_df = pd.read_csv('output_data/weather_data.csv', index_col ='Unnamed: 0')
opened_df


,city,lat,long,temp,humidity,cloudiness,windspeed,max temp,country
0,Minas,-34.38,-55.24,8.57,76,75,3.60,9.00,UY
1,Rikitea,-23.12,-134.97,21.42,70,0,9.43,21.42,PF
2,Mayo,38.89,-76.51,29.61,48,1,2.10,31.11,US
3,Severo-Kuril'sk,50.68,156.12,8.09,89,92,3.74,8.09,RU
4,Rabo de Peixe,37.80,-25.58,19.00,77,75,1.00,19.00,PT
...,...,...,...,...,...,...,...,...,...
539,Nacala,-14.54,40.67,21.28,75,78,4.86,21.28,MZ
540,Ati,13.22,18.34,33.60,25,56,3.06,33.60,TD
541,Kingsport,36.55,-82.56,29.31,45,40,2.10,30.00,US
542,Sibu,2.30,111.82,23.93,88,40,1.50,24.00,MY


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
opened_df.dropna()

# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = opened_df[["lat", "long"]]

# set humidity values
humidity = opened_df["humidity"].astype(float)




In [32]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
# narrow by: temp between 70 and 80, windspeed less than 5, 0 cloudiness
# create new df, drop rest of rows
vacay_df = opened_df.loc[(opened_df["max temp"] > 25) & (opened_df["max temp"] < 30) & (opened_df["windspeed"] < 10) & (opened_df["cloudiness"] < 10)]
vacay_df

,city,lat,long,temp,humidity,cloudiness,windspeed,max temp,country
29,Kapaa,22.08,-159.32,28.42,62,1,6.20,29.44,US
88,Sur,22.57,59.53,29.19,80,1,1.55,29.19,OM
108,Jamestown,42.10,-79.24,26.30,54,1,4.60,27.00,US
129,Myrtle Beach,33.69,-78.89,26.76,88,1,3.10,27.22,US
197,Hilo,19.73,-155.09,26.46,58,1,4.60,29.00,US
278,Umluj,25.02,37.27,26.79,52,0,1.09,26.79,SA
343,Khowst,33.38,69.88,25.52,21,0,2.60,25.52,AF
357,Dalbandin,28.89,64.41,28.10,22,0,2.18,28.10,PK
384,Grafton,42.21,-71.69,24.70,64,1,3.60,26.67,US
400,Alice Town,25.72,-79.30,29.94,66,8,4.09,29.94,BS


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = vacay_df
hotel_df["Hotel Name"] = ""
hotel_df

/Users/marksutton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,lat,long,temp,humidity,cloudiness,windspeed,max temp,country,Hotel Name
29,Kapaa,22.08,-159.32,28.42,62,1,6.20,29.44,US,
88,Sur,22.57,59.53,29.19,80,1,1.55,29.19,OM,
108,Jamestown,42.10,-79.24,26.30,54,1,4.60,27.00,US,
129,Myrtle Beach,33.69,-78.89,26.76,88,1,3.10,27.22,US,
197,Hilo,19.73,-155.09,26.46,58,1,4.60,29.00,US,
278,Umluj,25.02,37.27,26.79,52,0,1.09,26.79,SA,
343,Khowst,33.38,69.88,25.52,21,0,2.60,25.52,AF,
357,Dalbandin,28.89,64.41,28.10,22,0,2.18,28.10,PK,
384,Grafton,42.21,-71.69,24.70,64,1,3.60,26.67,US,
400,Alice Town,25.72,-79.30,29.94,66,8,4.09,29.94,BS,


In [35]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["long"]
    #print(lat)
    #print(lng)

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

/Users/marksutton/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.


,city,lat,long,temp,humidity,cloudiness,windspeed,max temp,country,Hotel Name
29,Kapaa,22.08,-159.32,28.42,62,1,6.20,29.44,US,Sheraton Kauai Resort at Coconut Beach
88,Sur,22.57,59.53,29.19,80,1,1.55,29.19,OM,Sur Plaza Hotel
108,Jamestown,42.10,-79.24,26.30,54,1,4.60,27.00,US,Hampton Inn & Suites Jamestown
129,Myrtle Beach,33.69,-78.89,26.76,88,1,3.10,27.22,US,Bay View Resort Myrtle Beach
197,Hilo,19.73,-155.09,26.46,58,1,4.60,29.00,US,Hilo Hawaiian Hotel
278,Umluj,25.02,37.27,26.79,52,0,1.09,26.79,SA,Moon light Furnished Units
343,Khowst,33.38,69.88,25.52,21,0,2.60,25.52,AF,وچه خوړه،Wacha Khwara
357,Dalbandin,28.89,64.41,28.10,22,0,2.18,28.10,PK,Gidaan Guest House
384,Grafton,42.21,-71.69,24.70,64,1,3.60,26.67,US,The Grafton Inn
400,Alice Town,25.72,-79.30,29.94,66,8,4.09,29.94,BS,Bimini Big Game Club Resort & Marina


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_hotel = hotel_df[["lat", "long"]]
# print(hotel_info)

In [37]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
marker_layer = gmaps.marker_layer(locations_hotel, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))